<a href="https://colab.research.google.com/github/sandroklippel/misc_noteboooks/blob/main/Historico_Mapbiomas_de_um_imovel_CAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Histórico de Uso e Cobertura do Solo de um imóvel registrado no CAR

Instruções:

1.   Atribua o código do imóvel na variável `cod_imovel` abaixo (exemplo `cod_imovel='PR-4113700-F29A0388E6D44ACE91A01855F5076D67'`). O polígono do  imóvel será obtido diretamente do servidor WFS do SICAR.
2.   Atribua o nome do projeto registrado no Google Earth Engine na variável `PROJECT`. Você encontra o nome do projeto em code.earthengine.google.com como no exemplo abaixo: 
![image.png](image.png)
3. Selecione no menu "Ambiente de execução" -> "Executar tudo" ou pressione Ctrl+F9
4. O gráfico será gerado na última célula. Há opção para salvá-lo em .PNG.



In [ ]:
cod_imovel='PR-4113700-F29A0388E6D44ACE91A01855F5076D67'
PROJECT=''

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project=PROJECT)

In [ ]:
import datetime
import json
import ssl
import pandas as pd
import numpy as np
import plotly.express as px
from urllib import request
from urllib.parse import urlencode
from shapely.geometry import shape

def busca_imovel(cod_imovel):
    """
    Busca o poligono do imóvel no servidor WFS do CAR
    """
    uf = cod_imovel.split('-')[0].lower()
    typename = f'sicar:sicar_imoveis_{uf}'
    
    filter = f'''
    <Filter>
        <PropertyIsEqualTo>
        <PropertyName>cod_imovel</PropertyName>
            <Literal>{cod_imovel}</Literal>
        </PropertyIsEqualTo>
    </Filter>'''
    
    query = urlencode({'service': 'WFS',
                'version': '1.0.0',
                'request': 'GetFeature',
                'typeName': typename,
                'FILTER': filter,
                'srsname': 'EPSG:4326',
                'outputFormat': 'application/json'})
    
    ctx = ssl.create_default_context()
    ctx.set_ciphers('DEFAULT')
    
    endpoint = 'https://geoserver.car.gov.br/geoserver/sicar/ows?'
    url = endpoint + query
    
    with request.urlopen(url, context=ctx) as res:
        content = res.read()
        return json.loads(content)

def map_function(s):
  def mbyr(y):
      k = ee.String('classification_').cat(ee.String(y))
      d = ee.Dictionary(s.get(k))
      return d.set('year', y)
  return mbyr

classes = {1: 'Floresta',
           3: 'Formação Florestal',
           4: 'Formação Savânica',
           5: 'Mangue',
           6: 'Floresta Alagável',
           9: 'Silvicultura',
           10: 'Vegetação Herbácea e Arbustiva',
           11: 'Campo Alagado e Área Pantanosa',
           12: 'Formação Campestre',
           14: 'Agropecuária',
           15: 'Pastagem',
           18: 'Agricultura',
           19: 'Lavoura Temporária',
           20: 'Cana',
           21: 'Mosaico de Usos',
           22: 'Área não Vegetada',
           23: 'Praia, Duna e Areal',
           24: 'Área Urbanizada',
           25: 'Outras Áreas não Vegetadas',
           26: "Corpo D'água",
           27: 'Não observado',
           29: 'Afloramento Rochoso',
           30: 'Mineração',
           31: 'Aquicultura',
           32: 'Apicum',
           33: 'Rio, Lago e Oceano',
           35: 'Dendê',
           36: 'Lavoura Perene',
           39: 'Soja',
           40: 'Arroz',
           41: 'Outras Lavouras Temporárias',
           46: 'Café',
           47: 'Citrus',
           48: 'Outras Lavouras Perenes',
           49: 'Restinga Arbórea',
           50: 'Restinga Herbácea',
           62: 'Algodão (beta)',
           0: 'Não observado'}

cores =  {1: '#32a65e',
          3: '#1f8d49',
          4: '#7dc975',
          5: '#04381d',
          6: '#026975',
          9: '#7a5900',
          10: '#ad975a',
          11: '#519799',
          12: '#d6bc74',
          14: '#FFFFB2',
          15: '#edde8e',
          18: '#E974ED',
          19: '#C27BA0',
          20: '#db7093',
          21: '#ffefc3',
          22: '#d4271e',
          23: '#ffa07a',
          24: '#d4271e',
          25: '#db4d4f',
          26: '#0000FF',
          27: '#ffffff',
          29: '#ffaa5f',
          30: '#9c0027',
          31: '#091077',
          32: '#fc8114',
          33: '#2532e4',
          35: '#9065d0',
          36: '#d082de',
          39: '#f5b3c8',
          40: '#c71585',
          41: '#f54ca9',
          46: '#d68fe2',
          47: '#9932cc',
          48: '#e6ccff',
          49: '#02d659',
          50: '#ad5100',
          62: '#ff69b4',
          0:  '#ffffff'}

color_map = {classes[k]: v for k, v in cores.items()}

feature = busca_imovel(cod_imovel)

hoje = datetime.datetime.today().strftime('%d/%m/%Y')
geom = ee.FeatureCollection(feature).geometry()
area_ha = geom.area().getInfo() / 10000.0

mapbiomas = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1')

stats = (mapbiomas.reduceRegion(
         reducer = ee.Reducer.frequencyHistogram(),
        geometry = geom,
        scale = 30,
        crs = 'EPSG:4326'
        ))

years = ee.List([str(i) for i in range(1985,2024)])

map_by_year = map_function(stats)

lst = years.map(map_by_year)

df = pd.DataFrame(lst.getInfo())
df.set_index('year', inplace=True)
df_perc = df.div(df.iloc[0,:].sum(), axis=1).mul(100).round(2)
df_perc.columns = [classes[int(c)] for c in df_perc.columns.tolist()]
df_perc = df_perc.reindex(columns=sorted(df_perc.columns))

In [ ]:
fig = px.bar(df_perc, barmode='stack', 
             color_discrete_map=color_map, 
             labels={"value": "% da área do polígono", "variable": "", "year": ""},
             title=f"Histórico de Uso e Cobertura do Solo do imóvel {cod_imovel} (aprox. {area_ha:.2f} ha)")

fig.update_layout(template='simple_white')
fig.update_xaxes(tickangle=-90)
fig.add_annotation(
    text = f"Fonte dos dados: Projeto MapBiomas - Coleção 9 da Série Anual de Mapas de Uso e Cobertura da Terra do Brasil, acesso em {hoje}.",
    showarrow=False,
    x = 0,
    y = -0.2,
    xref='paper',
    yref='paper',
    xanchor='left',
    yanchor='bottom',
    xshift=-1,
    yshift=-5,
    font=dict(size=10, color="grey"),
    align="left")
fig.show()

### Polígono para simples conferência:

In [ ]:
shape(geom.getInfo())